# Particle Swarm Optimization

Sources:
- http://www.swarmintelligence.org/tutorials.php
- https://github.com/tisimst/pyswarm/blob/master/pyswarm/pso.py

At iteration $k$, particle $i$ has posistion $x^i_k$ defined recursively as:

$$x^i_{k+1} = x^i_k + v^i_{k+1}$$

where $v$ is its velocity defined as follows:

$$ v^i_{k+1} = w_k \cdot v^i_{k} + c_1 r_1 (p^i_k - x^i_k) + c_2 r_2 (p^g_k - x^i_k)$$

Where:

- $p^i_k$ is the particle's best known position
- $p^g_k$ is the global (all particles) best known position
- $w_k$ is an inertia parameter
- $c_1$ is the cognitive parameter
- $c_2$ is the social parameter
- $r_1, r_2$ are random numbers in $(0, 1)$

In plain English, the velocity update is defined as:

new_velocity = (inertia * previous_velocity) + (move in the direction of the personal best) + (move in the direction of the global best)

In [5]:
# import two useful libraries/modules
import numpy as np
import random

In [16]:
# -- global parameters
c1 = 1 # cognitive parameter
c2 = 2 # social parameter
w = .5 # inertia

# -- Particle Class

class Particle:
    """A Python Class for a simple particle."""
    
    def __init__(self, upper, lower, ndim):
        '''Initiate a Particle with an upper and lower bound, and a number of dimensions.'''
        # Initiate position and velocity randomly
        self.position = np.array([random.uniform(lower, upper) for _ in range(ndim)])
        self.velocity = np.array([random.uniform(-1, 1) for _ in range(ndim)])
        # These attributes are here to store the "memory" of the function
        self.personal_best_position = self.position # initial position
        self.personal_best_error = np.inf # infinity
    
    def __str__(self):
        '''what will be called when you print a Particle instance, could be useful for debugging.'''
        return("I am a particle with best position {},\ncurrent position {},\nand velocity {}.\n"
               .format(self.personal_best_position, self.position, self.velocity))       
    
    def current_error(self, function):
        '''Compute the current "fitness" value of a particle, given a function. Updates it if needed.'''
        current_fitness = function(self.position)
        if current_fitness < self.personal_best_error:
            self.personal_best_error = current_fitness
            self.personal_best_position = self.position
        return current_fitness
    
    def update_velocity(self, global_best):
        '''update a particle's velocity'''
        # two random coefficients in uniform([0, 1])
        r1 = random.random()
        r2 = random.random()
        self.velocity = (w*self.velocity + # inertia
                         c1*r1*(self.personal_best_position - self.position) + # cognitive term
                         c2*r2*(global_best - self.position)) # social term
            
    def move(self):
        '''Moves a Particle at a new iteration. Deals with out-of-bound cases (are they ok or not?)'''
        self.position = self.position + self.velocity
        # need to deal with when the particle goes out of bound...

In [17]:
# --- Solver Class
class PSO:    
    """Solver instance for Particle Swarm Optimizer. (Basically the mastermind which commands the Particles.)
    Initiated with:
    - num_particles: how many particles to use?
    - function: function to optimize.
    - n_iter: number of iterations to be performed.
    - lower, upper: lower and upper bounds for the search space (for now, a box i.e. they are the same accross all dimensions.
    - ndim: dimensionality of the search space.)"""
    
    def __init__(self, num_particles, function, n_iter, lower = -10, upper = 10, ndim = 3):
        '''Initiate the solver'''
        # create all the Particles, stored in a list.
        self.particles = [Particle(lower, upper, ndim) for _ in range(num_particles)]
        # store global best and associated error
        self.global_best = np.array([])
        self.global_best_error = np.inf # infinity
        self.function = function # function to be optimised
        self.n_iter = n_iter # num of iterations
        
    def update_best(self):
        '''Find the new best global position and update it.'''
        for particle in self.particles:
            # ask all particles for their current error. If better than current global best, update.
            if particle.current_error(self.function) < self.global_best_error:
                self.global_best = particle.position
                self.global_best_error = particle.current_error(self.function)
                
    def move_particles(self):
        '''Run one iteration of the algorithm. Update particles velocity and move them.'''
        for particle in self.particles:
            particle.update_velocity(self.global_best)
            particle.move()
    
    def __str__(self):
        '''Print best global position when calling `print(pso instance)`'''
        return """Current best position: {}
        With error: {}""".format(self.global_best, self.global_best_error)
    
    def go(self):
        '''Run the algorithm and print the result. By default, update us every 50 iterations.'''
        print("Running the PSO algorithm with {} particles, for {} iterations.\n".
              format(len(self.particles), self.n_iter))
        
        for iteration in range(self.n_iter):
            pso.update_best()
            pso.move_particles()
            if iteration % 50 == 0:
                print("Iteration number " + str(iteration))
                print("Current best error: " + str(self.global_best_error))
                print("\n")
                
        print("Found minimum at {} with value {}.".format(self.global_best, self.global_best_error))

### Tests

In [18]:
def sumsquares(x):
    return(sum([i**2 for i in x]))

In [19]:
pso = PSO(100, sumsquares, n_iter = 100)
pso.go()

Running the PSO algorithm with 100 particles, for 100 iterations.

Iteration number 0
Current best error: 4.594739406432228


Iteration number 50
Current best error: 7.316314611802258e-13


Found minimum at [9.84812004e-14 2.01980195e-12 1.37353660e-12] with value 5.975901241199298e-24.


In [20]:
for p in pso.particles[0:10]:
    print(p)

I am a particle with best position [4.77976268e-11 1.99249431e-11 1.70834536e-11],
current position [4.31224300e-11 2.45946751e-11 4.32396093e-12],
and velocity [-4.67519678e-12  4.66973196e-12 -1.27594927e-11].

I am a particle with best position [-3.92321597e-11  3.25889909e-11 -1.50936246e-10],
current position [-5.41697865e-11  4.04284939e-11 -2.13038411e-10],
and velocity [-1.03450363e-10  7.27683625e-11 -4.09235647e-10].

I am a particle with best position [-1.08947212e-10  1.22262911e-10  1.98455429e-11],
current position [ 1.20091006e-10 -1.32812557e-10 -1.49164228e-11],
and velocity [ 4.20799832e-10 -4.77166174e-10 -5.01952244e-11].

I am a particle with best position [-4.81829864e-11 -9.39308042e-11 -9.63501046e-11],
current position [4.39943953e-10 4.00417610e-10 4.70771021e-11],
and velocity [6.49810137e-10 6.06588357e-10 1.05572238e-10].

I am a particle with best position [-2.60173804e-12  3.03944571e-12  3.21151772e-12],
current position [2.25182308e-11 1.87218555e-11 1.

In [21]:
def shiftedsumsquares(x):
    return(sum([(i+2)**2 for i in x])) # minimum should be all -2

In [22]:
pso = PSO(20, shiftedsumsquares, n_iter = 100) # works well even with 20 swarms!
pso.go()

Running the PSO algorithm with 20 particles, for 100 iterations.

Iteration number 0
Current best error: 5.1675198989926985


Iteration number 50
Current best error: 7.310353009112931e-12


Found minimum at [-2. -2. -2.] with value 3.390667823852566e-20.


Trying the function $f(x) := f(x_1, x_2) = (x_1 + 2 x_2 − 3)^2 + (x_1 − 2)^2$

In [24]:
def newfunct(x):
    return (x[0] + 2*x[1] - 3)**2 + (x[0] - 2)**2

In [25]:
pso = PSO(100, newfunct, 100, ndim=2)
pso.go()

Running the PSO algorithm with 100 particles, for 100 iterations.

Iteration number 0
Current best error: 0.5812078181485358


Iteration number 50
Current best error: 5.601709483878306e-17


Found minimum at [2.  0.5] with value 3.743144995273701e-26.


## Parallelizing

In [ ]:
#...